<a href="https://colab.research.google.com/github/subham129/Deep-learning/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Upload kaggle.json
from google.colab import files
files.upload()

# Step 2: Move kaggle.json to the correct directory and set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 3: Download the Fake News dataset
!kaggle competitions download -c fake-news

# Step 4: Unzip the dataset
!unzip fake-news.zip

# Step 5: Load the dataset into pandas
import pandas as pd
df = pd.read_csv('train.csv')  # Adjust file name if necessary
df.head()  # Display the first few rows


Saving kaggle.json to kaggle (1).json
 99% 46.0M/46.5M [00:04<00:00, 14.9MB/s]
100% 46.5M/46.5M [00:04<00:00, 11.2MB/s]
Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.shape

(20800, 5)

In [3]:
df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [4]:
df.dropna(inplace=True)

In [5]:
x = df.drop('label',axis=1)
y = df['label']

In [6]:
x.shape

(18285, 4)

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
voc_size=5000

In [11]:
messages=x.copy()

In [12]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [13]:
messages.reset_index(inplace=True)

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
import nltk
import re

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1537, 4409, 2340, 2276, 2795, 3023, 4273, 2769, 3891, 4872],
 [232, 3977, 3783, 4263, 97, 1943, 3993],
 [3801, 4743, 4385, 3654],
 [1321, 2955, 1998, 3001, 79, 4967],
 [3176, 97, 847, 4980, 3330, 2565, 97, 3829, 259, 3921],
 [3313,
  2529,
  1079,
  976,
  87,
  3444,
  921,
  1139,
  4651,
  1728,
  3717,
  136,
  3724,
  944,
  3993],
 [971, 1663, 2438, 4707, 2097, 632, 981, 3394, 4358, 3220, 992],
 [4952, 4145, 900, 2840, 3201, 2587, 3444, 4266, 4358, 3220, 992],
 [1284, 4966, 3156, 70, 2194, 2332, 2848, 1570, 3444, 1314],
 [2563, 11, 2784, 3623, 2906, 4791, 562, 1394],
 [2814, 4443, 1939, 4891, 2880, 1309, 4274, 3539, 3397, 2634, 1555],
 [3001, 4422, 2795, 2332, 3444, 3201],
 [4499, 4702, 2827, 1320, 1408, 2690, 4110, 3391, 3527],
 [2905, 2855, 4159, 1174, 2173, 2811, 1651, 4358, 3220, 992],
 [53, 733, 405, 2148, 674, 4358, 3220, 992],
 [1048, 4122, 3669, 3609, 1550, 108, 880, 1508, 4041, 1973],
 [873, 3046, 3977],
 [2566, 4574, 869, 3188, 3444, 1626, 1864, 3993],
 [4131, 106, 37

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2769 3891 4872]
 [   0    0    0 ...   97 1943 3993]
 [   0    0    0 ... 4743 4385 3654]
 ...
 [   0    0    0 ... 4358 3220  992]
 [   0    0    0 ... 2571 3000 4680]
 [   0    0    0 ... 2190 3021 3357]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1537,
       4409, 2340, 2276, 2795, 3023, 4273, 2769, 3891, 4872], dtype=int32)

In [27]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7970 - loss: 0.4142 - val_accuracy: 0.9133 - val_loss: 0.1954
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9470 - loss: 0.1345 - val_accuracy: 0.9246 - val_loss: 0.1981
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9675 - loss: 0.0900 - val_accuracy: 0.9211 - val_loss: 0.2302
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9763 - loss: 0.0705 - val_accuracy: 0.9191 - val_loss: 0.2668
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9868 - loss: 0.0442 - val_accuracy: 0.9183 - val_loss: 0.2400
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9913 - loss: 0.0332 - val_accuracy: 0.9105 - val_loss: 0.3531
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9947 - loss: 0.0182 - val_accuracy: 0.9157 - val_loss: 0.3903
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9972 - loss: 0.0122 - val_accuracy: 0

In [42]:
yp = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [44]:

y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [49]:
y_pred[:10]

[1, 1, 0, 0, 1, 0, 0, 1, 1, 0]

In [50]:
y_test[:10]

array([1, 0, 0, 0, 1, 1, 0, 1, 1, 1])

In [47]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

-1.6800918236192235

In [33]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [39]:
y_pred = (model.predict(X_test))

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
